Data cleaning is definitely a "practice makes perfect" skill. Using this
dataset of article open-access prices paid by the WELLCOME Trust between 
2012 and 2013, determine the five most common journals and the total articles
for each. Next, calculate the mean, median, and standard deviation of the 
open-access cost per article for each journal . You will need to do 
considerable data cleaning in order to extract accurate estimates, and may 
want to look into data encoding methods if you get stuck. For a real bonus 
round, identify the open access prices paid by subject area.

As noted in the previous assignment, don't modify the data directly. Instead, 
write a cleaning script that will load the raw data and whip it into shape. 
Jupyter notebooks are a great format for this. Keep a record of your decisions:
well-commented code is a must for recording your data cleaning decision-making 
progress. Submit a link to your script and results below and discuss it with 
your mentor at your next session.

In [1]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#import data set
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='Latin-1')

In [3]:
#show data
df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [4]:
#renaming ugly column name
price = df.iloc[:,-1]
df["Price"] = price
df.columns
df = df.drop('COST (£) charged to Wellcome (inc VAT when charged)', axis = 1)

In [5]:
#show data
df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,Price
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,PMC3579457,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,PMC3709265,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,23057412 PMC3495574,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,PMCID: PMC3780468,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,PMCID: PMC3621575,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [6]:
#Making everything uppercase to consolidate some journals
df = df.apply(lambda x: x.astype(str).str.upper())

In [7]:
# words to remove in order to consolidate further
remove_words = ['JOURNAL', 'JNL', 'JOURNALS' 'FOR', 'OF', 'PART A', '&', 'J ', 
                 '.', 'THE']

remove_Words = ['FOR', 'AND', '&','THE', 'OF', 'LTD', 'LIMITED', 'PUBLISHING', 
          'GROUP', 'JOURNAL SERVICES', 'MEDIA', 'SA', 'CORPORATION', 'LLC']

In [8]:
#Removing from Journal Title column
pat = r'\b(?:{})\b'.format('|'.join(remove_words))
df['Journal title'] = df['Journal title'].str.replace(pat, '')


In [9]:
#Removing from Publisher Column
PAT = r'\b(?:{})\b'.format('|'.join(remove_Words))
df['Publisher'] = df['Publisher'].str.replace(PAT, '')

In [10]:
#removing $ and £
df['Price'] = df['Price'].replace({'\$':''}, regex = True)
df['Price'] = df['Price'].replace({'£':''}, regex = True)

In [11]:
#converting back to numeric
df['Price'] = pd.to_numeric(df['Price'])

In [12]:
#Fixing outliers
for i in range(2127):
    if df['Price'][i] > 20000:
        df['Price'][i] = None
        #print(df['Publisher'][i])
    elif df['Price'][i] == 0:
        df['Price'][i] = None
        #print(df['Publisher'][i])

for i in range(2127):
    if np.isnan(df['Price'][i]) == True:
        df['Price'][i] = df[df['Publisher'] == '{}'.format(df['Publisher'][i])].mean(skipna = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [14]:
#show cleaned data
df.head(10)

,PMID/PMCID,Publisher,Journal title,Article title,Price
0,NAN,CUP,PSYCHOLOGICALMEDICINE,REDUCED PARAHIPPOCAMPAL CORTICAL THICKNESS IN ...,2034.00
1,PMC3679557,ACS,BIOMACROMOLECULES,STRUCTURAL CHARACTERIZATION OF A MODEL GRAM-NE...,2381.04
2,23043264 PMC3506128,ACS,MEDCHEM,"FUMAROYLAMINO-4,5-EPOXYMORPHINANS AND RELATED ...",642.56
3,23438330 PMC3646402,ACS,MEDCHEM,ORVINOLS WITH MIXED KAPPA/MU OPIOID RECEPTOR A...,669.64
4,23438216 PMC3601604,ACS,ORGCHEM,REGIOSELECTIVE OPENING OF MYO-INOSITOL ORTHOES...,685.88
5,PMC3579457,ACS,MEDICINALCHEMISTRY,COMPARATIVE STRUCTURAL AND FUNCTIONAL STUDIES ...,2392.20
6,PMC3709265,ACS,PROTEOMERESEARCH,MAPPING PROTEOLYTIC PROCESSING IN THE SECRETOM...,2367.95
7,23057412 PMC3495574,ACS,MOLPHARM,QUANTITATIVE SILENCING OF EGFP REPORTER GENE B...,649.33
8,PMCID: PMC3780468,ACS (AMERCIAN CHEMICAL SOCIETY) PUBLICATIONS,ACSCHEMICALBIOLOGY,A NOVEL ALLOSTERIC INHIBITOR OF THE URIDINE DI...,1294.59
9,PMCID: PMC3621575,ACS (AMERCIAN CHEMICAL SOCIETY) PUBLICATIONS,ACSCHEMICALBIOLOGY,CHEMICAL PROTEOMIC ANALYSIS REVEALS THE DRUGAB...,1294.78


In [15]:
#Find the top 5 journals
Top5_Journals = df["Journal title"].value_counts()[0:5]
print(Top5_Journals)

PLOSONE                199
BIOLOGICALCHEMISTRY     65
NEUROIMAGE              29
PLOSPATHOGENS           24
PLOSGENETICS            24
Name: Journal title, dtype: int64


In [17]:
#Values for PLOSONE
print("mean: ", df[df['Journal title'] =='PLOSONE'].mean())
print("median: ", df[df['Journal title'] =='PLOSONE'].median())
print("Std. Dev. : ", df[df['Journal title'] =='PLOSONE'].std())

mean:  Price    941.318893
dtype: float64
median:  Price    897.19
dtype: float64
Std. Dev. :  Price    198.257064
dtype: float64


In [18]:
#Values for BIOLOGICAL CHEMISTRY
print("mean: ", df[df['Journal title'] =='BIOLOGICALCHEMISTRY'].mean())
print("median: ", df[df['Journal title'] =='BIOLOGICALCHEMISTRY'].median())
print("Std. Dev. : ", df[df['Journal title'] =='BIOLOGICALCHEMISTRY'].std())

mean:  Price    1379.445528
dtype: float64
median:  Price    1299.74
dtype: float64
Std. Dev. :  Price    405.838863
dtype: float64


In [19]:
#Values for NEUROIMAGE
print("mean: ", df[df['Journal title'] =='NEUROIMAGE'].mean())
print("median: ", df[df['Journal title'] =='NEUROIMAGE'].median())
print("Std. Dev. : ", df[df['Journal title'] =='NEUROIMAGE'].std())

mean:  Price    2215.168276
dtype: float64
median:  Price    2326.43
dtype: float64
Std. Dev. :  Price    266.653947
dtype: float64


In [20]:
#Values for PLOSGENETICS
print("mean: ", df[df['Journal title'] =='PLOSGENETICS'].mean())
print("median: ", df[df['Journal title'] =='PLOSGENETICS'].median())
print("Std. Dev. : ", df[df['Journal title'] =='PLOSGENETICS'].std())

mean:  Price    1601.184905
dtype: float64
median:  Price    1696.67
dtype: float64
Std. Dev. :  Price    204.089235
dtype: float64


In [ ]:
#Values for PLOSPATHOGENS
print("mean: ", df[df['Journal title'] =='PLOSPATHOGENS'].mean())
print("median: ", df[df['Journal title'] =='PLOSPATHOGENS'].median())
print("Std. Dev. : ", df[df['Journal title'] =='PLOSPATHOGENS'].std())